In [34]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
import math

from sympy import symbols, Matrix, log, sqrt
from project_dir import project_dir
Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()

k = Eigen.Scalar('k')
L0 = Eigen.Scalar("L0")
radius = Eigen.Scalar("r")
_pi = Eigen.Scalar("pi")

X = Eigen.Vector("X", 9)
X0 = X[0:3,0]
X1 = X[3:6,0]
X2 = X[6:9,0]

e0 = X1 - X0
e1 = X2 - X1

In [35]:

kappa = e0.cross(e1) * 2 / (sqrt(e0.dot(e0) * e1.dot(e1)) + e0.dot(e1))
kappa

Matrix([
[ (2*(-X(1) + X(4))*(-X(5) + X(8)) - 2*(-X(2) + X(5))*(-X(4) + X(7)))/(sqrt(((-X(0) + X(3))**2 + (-X(1) + X(4))**2 + (-X(2) + X(5))**2)*((-X(3) + X(6))**2 + (-X(4) + X(7))**2 + (-X(5) + X(8))**2)) + (-X(0) + X(3))*(-X(3) + X(6)) + (-X(1) + X(4))*(-X(4) + X(7)) + (-X(2) + X(5))*(-X(5) + X(8)))],
[(-2*(-X(0) + X(3))*(-X(5) + X(8)) + 2*(-X(2) + X(5))*(-X(3) + X(6)))/(sqrt(((-X(0) + X(3))**2 + (-X(1) + X(4))**2 + (-X(2) + X(5))**2)*((-X(3) + X(6))**2 + (-X(4) + X(7))**2 + (-X(5) + X(8))**2)) + (-X(0) + X(3))*(-X(3) + X(6)) + (-X(1) + X(4))*(-X(4) + X(7)) + (-X(2) + X(5))*(-X(5) + X(8)))],
[ (2*(-X(0) + X(3))*(-X(4) + X(7)) - 2*(-X(1) + X(4))*(-X(3) + X(6)))/(sqrt(((-X(0) + X(3))**2 + (-X(1) + X(4))**2 + (-X(2) + X(5))**2)*((-X(3) + X(6))**2 + (-X(4) + X(7))**2 + (-X(5) + X(8))**2)) + (-X(0) + X(3))*(-X(3) + X(6)) + (-X(1) + X(4))*(-X(4) + X(7)) + (-X(2) + X(5))*(-X(5) + X(8)))]])

In [36]:
alpha = k * radius**4 * _pi / 4
alpha

Matrix([[k*pi*xi**4/64]])

In [37]:
E = alpha * kappa.dot(kappa) / L0
E

Matrix([[k*pi*xi**4*((2*(-X(0) + X(3))*(-X(4) + X(7)) - 2*(-X(1) + X(4))*(-X(3) + X(6)))**2/(sqrt(((-X(0) + X(3))**2 + (-X(1) + X(4))**2 + (-X(2) + X(5))**2)*((-X(3) + X(6))**2 + (-X(4) + X(7))**2 + (-X(5) + X(8))**2)) + (-X(0) + X(3))*(-X(3) + X(6)) + (-X(1) + X(4))*(-X(4) + X(7)) + (-X(2) + X(5))*(-X(5) + X(8)))**2 + (-2*(-X(0) + X(3))*(-X(5) + X(8)) + 2*(-X(2) + X(5))*(-X(3) + X(6)))**2/(sqrt(((-X(0) + X(3))**2 + (-X(1) + X(4))**2 + (-X(2) + X(5))**2)*((-X(3) + X(6))**2 + (-X(4) + X(7))**2 + (-X(5) + X(8))**2)) + (-X(0) + X(3))*(-X(3) + X(6)) + (-X(1) + X(4))*(-X(4) + X(7)) + (-X(2) + X(5))*(-X(5) + X(8)))**2 + (2*(-X(1) + X(4))*(-X(5) + X(8)) - 2*(-X(2) + X(5))*(-X(4) + X(7)))**2/(sqrt(((-X(0) + X(3))**2 + (-X(1) + X(4))**2 + (-X(2) + X(5))**2)*((-X(3) + X(6))**2 + (-X(4) + X(7))**2 + (-X(5) + X(8))**2)) + (-X(0) + X(3))*(-X(3) + X(6)) + (-X(1) + X(4))*(-X(4) + X(7)) + (-X(2) + X(5))*(-X(5) + X(8)))**2)/(64*L0)]])

In [38]:
dEdX = VecDiff(E, X)
ddEddX = VecDiff(dEdX, X)

In [39]:
Cl = Gen.Closure(k, X, L0, radius, _pi)

In [40]:
s = f'''
{Cl("E",E)}
{Cl("dEdX",dEdX)}
{Cl("ddEddX",ddEddX)}
'''
print(s)

f = open( project_dir() / 'src/uipc/backends/cuda/finite_element/constitutions/sym/kirchhoff_rod_bending.inl', 'w')
f.write(s)
f.close()


template <typename T>
__host__ __device__ void E(T& R, const T& k, const Eigen::Vector<T,9>& X, const T& L0, const T& xi, const T& pi)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
Symbol Name Mapping:
k:
    -> {}
    -> Matrix([[k]])
X:
    -> {}
    -> Matrix([[X(0)], [X(1)], [X(2)], [X(3)], [X(4)], [X(5)], [X(6)], [X(7)], [X(8)]])
L0:
    -> {}
    -> Matrix([[L0]])
xi:
    -> {}
    -> Matrix([[xi]])
pi:
    -> {}
    -> Matrix([[pi]])
*****************************************************************************************************************************/
/* Sub Exprs */
auto x0 = -X(4) + X(7);
auto x1 = -X(0) + X(3);
auto x2 = 2*x1;
auto x3 = -X(